# **Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

# **Aprendizaje Supervisado**

## Entregable: MODELO DE CLASIFICACIÓN PARTE 1

## GRUPO Nº24

## INTEGRANTES:
   - Clarisa Manzone
   - Nico Rosales
   - Daniel Rubio
   - Diana Fonnegra
   ------

## Modelos de clasificación
 - RandomForest
-------

In [23]:
# Importación de modulos

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pprint import pprint

from scipy.stats import norm

from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder



from sklearn.decomposition import PCA

from sklearn.model_selection import cross_val_score, cross_validate, KFold, StratifiedKFold, \
    LeaveOneOut, LeavePOut,train_test_split, GridSearchCV, StratifiedKFold

from sklearn.metrics import recall_score, accuracy_score, precision_score, \
    f1_score, roc_auc_score, mean_squared_error, r2_score, confusion_matrix, \
    classification_report, ConfusionMatrixDisplay, accuracy_score, ConfusionMatrixDisplay, \
    classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC

import missingno as msno

sns.set_context('talk')
warnings.filterwarnings("ignore")

### Lectura del dataset

In [6]:
# Carga de los datasets (train y test)
train_titanic = pd.read_csv('CK_train.csv')
test_titanic = pd.read_csv('CK_test.csv')

### Extracción y resguardo de las etiquetas del set de entrenamiento

In [7]:
y = train_titanic['Transported']
y.replace({False: 0, True: 1},inplace=True)

### Lectura del dataset con los datos curados (Entrenamiento y prueba)
 - En el dataframe g_dataset cargamos los datos curados
 - IMPORTANTE: No confundir g_dataset con el homonimo de etapas previas de estudio(ej: EDA y curado).
 - g_dataset contiene datos preparados para aplicar directamente sobre el desarrollo de los modelos de clasificación

In [12]:
g_dataset = pd.read_csv('g_dataset_c.csv')
g_dataset.shape

(8693, 36)

### Separacion de los grupo de entrenamiento y prueba globales.
 - X representa el set de entrenamiento global.
 - A partir del mismo se generará el conjunto de entrenamiento y test para el desarrollo de los diferentes modelos.

In [13]:
X = g_dataset[: len(train_titanic)]
X_to_predict = g_dataset[len(train_titanic):]

# El set de datos disponible para el estudio y desarrollo de los modelos de clasificación
X.shape, y.shape

((7823, 36), (7823,))

In [14]:
# Los datos a predecir y realizar el submmit a Kaggle son:
X_to_predict.shape

(870, 36)

### Planteo General de distintos modelos de clasificación

In [15]:
clfs =  [DecisionTreeClassifier(random_state=10),
        RandomForestClassifier(random_state=10),
        LogisticRegression(random_state=10),
        MLPClassifier(random_state=10),
        XGBClassifier(random_state=10),
        GaussianNB(),
        LinearSVC(random_state=10),
        SGDClassifier(random_state=10)]

names = ['Arbol de decisión',
        'Random Forest',
        'Regresión Logística',
        'Perceptrón multicapa',
        'XGBoost',
        'Naive Bayes',
        'Suport Vector Machine',
        'Stochastic Gradient Descent']

trained_models = []
accuracy_models = []


#### Generación de los datos de entrenamiento y prueba a fin de estudiar los distintos modelos

In [17]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=10)

### Estudio del perfomance de los distintos modelos

In [19]:
from tabulate import tabulate

list_report = []

for clf, name in zip(clfs, names):

    clf.fit(X_train, y_train)

    train_predictions = clf.predict(X_train)
    accuracy_train = round( accuracy_score(y_train, train_predictions) * 100.0, 2)

    test_predictions = clf.predict(X_test)
    accuracy_test = round(accuracy_score(y_test, test_predictions) * 100.0, 2)

    list_report.append([name, accuracy_train, accuracy_test])

print(tabulate(list_report, headers=["Modelo","Acc Train [%]", "Acc Test [%]"]))

Modelo                         Acc Train [%]    Acc Test [%]
---------------------------  ---------------  --------------
Arbol de decisión                      99.97           75.59
Random Forest                          99.97           81.85
Regresión Logística                    78.68           80.26
Perceptrón multicapa                   67.18           65.75
XGBoost                                95.16           81.92
Naive Bayes                            76.08           77
Suport Vector Machine                  50.69           50.54
Stochastic Gradient Descent            78.41           80


### Seleccionamos Random Forest para profundizar su desarrollo

In [20]:
clf=RandomForestClassifier(random_state=10).fit(X_train, y_train)

In [21]:
test_predictions = clf.predict(X_test)
accuracy_test = round(accuracy_score(y_test, test_predictions) * 100.0, 2)
accuracy_test

81.85

In [22]:
print('Los parametros del modelo bajo análisis son:\n')
pprint(clf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 10,
 'verbose': 0,
 'warm_start': False}


### Buscando los mejores hiperparámetros del modelo...

### Busqueda aleatoria con validación cruzada

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Número de arboles
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Número de atributos a considerar en cada split
max_features = ['auto', 'sqrt']

# Máximo número de niveles en el arbol
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Mínimo número de muestras requeridas para hacer split de un nodo
min_samples_split = [2, 5, 10]

# Mínimo número de muestras requeridas por hoja
min_samples_leaf = [1, 2, 4]

# Método de selección de muestras para el entrenamiento de cada árbol
bootstrap = [True, False]



# Creación de la grilla aleatoria
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

### Acondicionamiento e inicio del proceso de búsqueda
 - Importante: El proceso toma mucho tiempo (horas), según la configuración del los parámetros de búsqueda

In [ ]:

# Usamos la grilla aleatoria para buscar los mejores hiperparámetros
# Modelo base a ajustar
rf = RandomForestRegressor()

# Búsqueda aleatoria de parámetros, usando validación cruzada de 3, entre 100 combinaciones diferentes y  todos los núcleos disponibles
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=10, random_state=10, n_jobs = -1)

# A buscar.... toma aproximandamente 2 a 3 hs dependiendo de la computadora usada
rf_random.fit(X_train, y_train)




### Los mejores paramatros obtenidos son:

In [26]:
rf_random.best_params_

NameError: name 'rf_random' is not defined

In [661]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    predictions = np.where(predictions > 0.5, 1, 0)
    print("Pred:",predictions[:10])
    print("TEST:",test_labels.values[:10])
    errors = abs(predictions - test_labels.values)
    print("error",errors[:10])
    mape = 100 * np.mean(errors)
    print(mape)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10, random_state = 10)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)


Pred: [0 0 1 1 0 0 0 0 1 0]
TEST: [0 0 1 1 0 0 1 0 1 0]
error [0 0 0 0 0 0 1 0 0 0]
19.297124600638977
Model Performance
Average Error: 0.1930 degrees.
Accuracy = 80.70%.


In [662]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)
y_test

Pred: [0 0 1 1 0 0 0 0 1 0]
TEST: [0 0 1 1 0 0 1 0 1 0]
error [0 0 0 0 0 0 1 0 0 0]
17.316293929712458
Model Performance
Average Error: 0.1732 degrees.
Accuracy = 82.68%.


2705    0
930     0
116     1
3467    1
838     0
       ..
22      0
3098    0
3223    1
2237    1
3159    0
Name: Transported, Length: 1565, dtype: int64

In [663]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of 2.45%.


### Optimización extra mediante ajuste adicional con grid search y validación cruzada
 - Se parte de los parametro obtenidos mediante busqueda aleatoria
 - Realizamos una busqueda en grilla alrededor de los mismos:
 -  'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 800

In [666]:
from sklearn.model_selection import GridSearchCV
# Creamos la grilla manteniendo los valores centrales de la misma con el resultado de la busqueda aleatoria
param_grid = {
    'bootstrap': [False],
    'max_depth': [20, 25, 30, 35, 40],
    'max_features': ['sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [3, 4, 5, 6, 7],
    'n_estimators': [700, 750, 800, 850, 9000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                          cv = 3, n_jobs = -1, verbose = 2)

In [668]:
# Ejecutamos la busqueda...
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 375 candidates, totalling 1125 fits


{'bootstrap': False,
 'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 6,
 'n_estimators': 700}

In [669]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

Pred: [0 0 1 1 0 0 0 0 1 0]
TEST: [0 0 1 1 0 0 1 0 1 0]
error [0 0 0 0 0 0 1 0 0 0]
16.996805111821086
Model Performance
Average Error: 0.1700 degrees.
Accuracy = 83.00%.


In [670]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

Improvement of 2.85%.


In [ ]:
best_grid.predict

In [672]:
test_predictions = best_grid.predict(X_to_predict)
test_predictions = np.where(test_predictions > 0.5, 1, 0)
test_predictions


array([1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,

In [673]:
submission = pd.DataFrame(list(zip(test_titanic["PassengerId"], test_predictions)),
                          columns=["PassengerId", "Transported"])
submission

,PassengerId,Transported
0,415,1
1,8663,1
2,4524,0
3,8246,0
4,2935,1
...,...,...
865,5996,0
866,5478,1
867,6760,1
868,1437,0


### Generamos la salida para Kaggle

In [674]:
submission.to_csv("submission_4.csv", header=True, index=False)

### Salvamos el modelo de mejor perfomance obtenido


In [679]:
import joblib
joblib.dump(best_grid, "./Bestrandom_forest.joblib")

['./Bestrandom_forest.joblib']

### Binarización de la salida

#### Importante al cargar el modelo en frio y producir la clasificación deberá aplicarse la binarización del dato de salida.
 - np.where(test_predictions > 0.5, 1, 0)

In [27]:
# loaded_rf = joblib.load("./Bestrandom_forest.joblib")
# test_predictions= loaded_rf.predict(X_prog)
# submission['Transported'] = np.where(test_predictions > 0.5, 1, 0)

———–FINAL PARTE 3 de 5————–